
All necessorry installation is done before executing the code

In [0]:
!pip install msgpack==0.5.6


import os
import nltk
from nltk.corpus import wordnet as wn
import re




import spacy
import spacy.cli
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES
from spacy import displacy



In [0]:
#download database
nltk.download('popular')


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [0]:
spacy.cli.download("en")
nlp = spacy.load("en")


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [0]:
def getNounChunks(doc):
  nounChunkList = []
  for chunk in doc.noun_chunks:
    nounChunkList.append([chunk.text, chunk.root.text, chunk.root.dep_,
          chunk.root.head.text])
  #for -ends
  return nounChunkList

In [0]:
# ------------------------- getNERFeatures() ---------------------------------|
def getNERFeatures(doc):  
  nerDict = {}
  for x in doc.ents:
    nerDict[x.text]=x.label_
  #for -ends
  return nerDict
# ------------------------- getNERFeatures() -ends ----------------------------|

In [0]:
# ------------------------- getTokens() ---------------------------------------|
def getTokens(sentence):
  '''3.1 Tokenize the FAQ and Answers into sentence and words'''
  tokens = nltk.word_tokenize(sentence)
  return (tokens)
# ------------------------- getTokens() -ends ---------------------------------|

In [0]:
# ------------------------- getFeatures() -------------------------------------|

#3.2 Lemmatize the words to extract the lemmas as feature
#3.3 Part-of-speech (POS) tag the words to extract POS tag features

def getFeatures(doc, featureList):
  for token in doc:
    if token.dep_ != 'punct' and token.dep_!='':
      featureDict = {}
      featureDict["token"] = token
      featureDict["lemmaVal"] = token.lemma
      featureDict["lemma"] = token.lemma_
      featureDict["pos"] = token.pos_
      featureDict["tag"] = token.tag_
      featureDict["shape"] = token.shape_
      featureDict["isAlpha"] = token.is_alpha
      featureDict["isStop"] = token.is_stop
      featureDict["dependency"]=token.dep_
      featureDict["headText"] = token.head.text
      featureDict["headTag"]=token.head.pos_
      featureDict["children"] = [child for child in token.children]
      featureList.append(featureDict)
  #for token -ends
  
  return featureList
# ------------------------- getFeatures() -ends--------------------------------|


In [0]:
# ------------------------- wordnetFeatures() ---------------------------------|

def wordnetFeatures(tokens):
  print(tokens, "\n")
  wordnetTokenDict = {}
  
  for token in tokens:
    tokenSynset = wn.synsets(token)
#     print("------------------------------------------------------------------")
#     print("token: {}".format(token))
#     print("tokenSynset: {}".format(tokenSynset))
    
    synonyms=[]
    hypernymList=[]
    hyponymList=[]
    holonymList=[]
    meronymList=[]
    entailList=[]
  
    tempTokenDict={}
    for index, tempSynset in enumerate(tokenSynset):
      
      hypernyms = tempSynset.hypernyms()
      for l in hypernyms:
        hypernymList.append(l.lemma_names()[0])
        
      hyponyms = tempSynset.hyponyms()
      for l in hyponyms:
        hyponymList.append(l.lemma_names()[0])
        
      holonyms = tempSynset.part_holonyms()
      for l in holonyms:
        holonymList.append(l.lemma_names()[0])
        
      meronyms = tempSynset.part_meronyms()
      for l in meronyms:
        meronymList.append(l.lemma_names()[0])
        
      entail = tempSynset.entailments()
      for l in entail:
        entailList.append(l.lemma_names()[0])
        
      for l in tempSynset.lemmas():
        synonyms.append(l.name())
     
#       print("current Synset: {}".format(tempSynset))
#       print("hypernyms: {}".format(hypernyms))
#       print("hyponyms: {}".format(hyponyms))
#       print("holonyms: {}".format(holonyms))
#       print("meronyms: {}".format(meronyms))
#       print("entail = {}".format(entail))
#       print("\n")
      
    tempTokenDict["hypernym"]=hypernymList  
    tempTokenDict["hyponym"]=hyponymList
    tempTokenDict["holonym"]=holonymList
    tempTokenDict["meronym"]=meronymList
    tempTokenDict["entail"]=entailList
    tempTokenDict["synonym"]=synonyms
    
    wordnetTokenDict[token]=tempTokenDict    
#     print("Synonym: {}".format(synonyms))
  
  return wordnetTokenDict
    
  #apply lesk...
# ------------------------- wordnetFeatures() ---------------------------------|

In [0]:
# ------------------------- acknowledgementTemplate() -------------------------|
def getAcknowledgementTemplate(doc, sentence):
  '''
  input: doc, featureList (List)
  output: ackTemplateDict (dictionary)
          Authors:
          paper:
          publication:
          publish year:
          reference page:
  '''
  NERDict = getNERFeatures(doc)
  publicationList = ["IEEE", "ACM","NIPS", "arXiv", "IBM", "dissertation", "Cambridge", "Univ." "Press", "McGraw-Hill", "Cornell university"]
  
  ackTemplateDict = {"authors":[], "paper":"", "publication": "", "publish-year":"", "reference-Page": ""}
  authorList = []
  for key in NERDict:
    if NERDict[key]=="PERSON":
      authorList.append(key)
    elif NERDict[key]=="DATE":
      ackTemplateDict["publish-year"]=key
    elif NERDict[key]=="CARDINAL":
      ackTemplateDict["reference-Page"]=key
    elif NERDict[key]=="ORG" or NERDict[key]=="GPE":
      ackTemplateDict["publication"]+=" "+key  
  #for key -ends
  
  for token in doc:
    for publication in publicationList:
      if publication.upper() in str(token).upper():
        ackTemplateDict["publication"]+=" "+str(token)  
      #if -ends
    #forr publication -ends
  #for token -ends
  
  paper=re.findall(r'“(.*?)”',sentence)
  if len(paper)!=0:
    ackTemplateDict["paper"]=paper[0]
  
    
  
  ackTemplateDict["authors"]=authorList
  return ackTemplateDict    
# ------------------------- acknowledgementTemplate() --------------------------|

In [0]:
def getDependencyTree(doc):
    nodes = []
    for token in doc:
        nodes.append([token.text, token.dep_, token.head.text, token.head.pos_,
          [child for child in token.children]])
    return nodes

In [0]:
def getAllChildren(sentence, tokenIndex):
  doc = nlp(sentence)
  return [token for token in doc[tokenIndex].children]

In [0]:
def printDependencies(sentence):
    nodes = get_dependency_tree_nodes(sentence)
    print("\n\n------------- Deps ------")
    for node in nodes:
        print(node)

In [0]:
def getVerbWithSubject(sentence):
  doc = nlp(sentence)
  verbs = set()
  for possible_subject in doc:
    if possible_subject.dep_ == 'nsubj' and possible_subject.head.pos == 'VERB':
      verbs.add(possible_subject.head)
  return (verbs)

In [0]:
def getLeftChildren(sentence, tokenIndex):
  doc = nlp(sentence)
  return [token for token in doc[tokenIndex].lefts]

In [0]:
def getRightChildren(sentence, tokenIndex):
  doc = nlp(sentence)
  return [token for token in doc[tokenIndex].rights]

In [0]:
def getRoot(sentence):
  doc = nlp(sentence)
  return [token for token in (doc) if token.head == token][0]

In [0]:
def getAncestors(sentence, descendant):
  doc = nlp(sentence)
  return  [ancestor for ancestor in descendant.ancestors]

In [0]:
def getPhraseSpan(sentence, tokenIndex):
  doc = nlp(sentence)
  span = doc[doc[tokenIndex].left_edge.i : doc[tokenIndex].right_edge.i+1]
  span.merge()
  return (span)

In [0]:
def getVerbSubject(sentence, tokenIndex):
  doc = nlp(sentence)
  return [token for token in doc[tokenIndex].children if token.dep_=="nsubj" or token.dep_=="nsubjpass"]

In [0]:
def getAllSubject(sentence):
  doc = nlp(sentence)
  sub =[]
  for token in doc:
    if token.dep_ == 'nsubj' or token.dep_=='nsubjpass':
      sub.append(token)
  return (sub)

In [0]:
def removeWords(query, removeWordList):
  querywords = query.split()  
  resultwords  = [word for word in querywords if word.lower() not in removeWordList]
  result = ' '.join(resultwords)
  return result

In [0]:
def getAllVerbs(sentence):
  doc = nlp(sentence)
  verbs =[]
  for token in doc:
    if token.pos_ == 'VERB':
      verbs.append(token)
  return (verbs)

In [0]:
def getTokenIndex(sentence, inToken):
  doc = nlp(sentence)

  tokenIndex = -1
  for index, token in enumerate(doc):
    if str(token)==str(inToken):
      return index
    #if -ends
  #for -ends
  return tokenIndex


In [0]:
def getTask(sentence, rootIndex):
  doc = nlp(sentence)
  #ckeft children for domain and task
  leftChildrenList = getLeftChildren(sentence, rootIndex)
  for childToken in leftChildrenList:
    #skip punct
    if childToken.dep_ != 'punct' and childToken.dep_!='':
      #search for nsubj or nsubjpass
      if childToken.dep_ == 'nsubj' or childToken.dep_ == 'nsubjpass':
        taskChildIndex = getTokenIndex(sentence, childToken)
        if taskChildIndex!=-1:
          taskSpan = getPhraseSpan(sentence, taskChildIndex)
          return taskSpan.text
      #if taskChildIndex -ends
  return ""

In [0]:
def getDomain(sentence, rootIndex):
  doc = nlp(sentence)
  #ckeft children for domain and task
  leftChildrenList = getLeftChildren(sentence, rootIndex)

  for childToken in leftChildrenList:
    #skip punct
    if childToken.dep_ != 'punct' and childToken.dep_!='':
      #look for prep
      if childToken.dep_=='prep':
        domainChildIndex = getTokenIndex(sentence, childToken)
        if domainChildIndex!=-1:
          domainSpan = getPhraseSpan(sentence, domainChildIndex)
          return domainSpan.text
  return ""

In [0]:
# ------------------------- getResultTemplate() -------------------------------|
def getResultTemplate(doc, sentence, featureList):
  '''
  input: doc, sentence, featureList (List)
  output: resultDict (dictionary)
          domain:
          task:
          attribute:
          result:
  '''
  #initializing dictionary
  resultDict = {"domain":"", "task": "", "result":""}
  
  #initializing indecator list
  resultVerbList = ["yield", "give", "render","return", "generate", "predict", "foretell", "produce", "lead", "forcast", "report", "achieve", "accomplish", "reach", "contain"]
  resultNounList = ["result", "outcome", "solution","effect", "decision", "answer", "final result", "score", "AP","precision", "recall", "accuracy", "fruit", "fruition", "r"]
  
  #check for indicator words
  positiveFlag = False
  for index, feature in enumerate(featureList):
    if (feature["lemma"] in resultVerbList) or (feature["lemma"] in resultVerbList):
      positiveFlag=True
    #if -ends
  #for -ends
  
  if not positiveFlag:
    return resultDict
  
  #get root
  rootToken = getRoot(sentence)
  verbSet = getVerbWithSubject(sentence)
  rootIndex = getTokenIndex(sentence, rootToken)
  
  #get right children of root
  rightChildrenList = getRightChildren(sentence, rootIndex)

  #result
  for childToken in rightChildrenList:
    if childToken.dep_ != 'punct' and childToken.dep_!='':
      if childToken.lemma_ in resultNounList:
        childIndex = getTokenIndex(sentence, childToken)
        span = getPhraseSpan(sentence, childIndex)
        #check for existance of NER in span
        resultDict["result"]=span
      #if -ends
    #if dep_ -ends
  #for -ends
  
  #ckeft children for domain and task
#   print("RootIndex_domain",rootIndex)
  domainSpan=getDomain(sentence, rootIndex)
#   print("RootIndex_Task",rootIndex)
  resultDict["domain"] = domainSpan
  taskSpan = getTask(sentence, rootIndex)
  resultDict["task"]=taskSpan
      
  return resultDict
# ------------------------- getResultTemplate() -ends--------------------------|

In [0]:
def getTerminologyTemplate(doc,sentence,featureList):
  resultTerminology={"key":"","definition":"","reference":""}
  doc=nlp(sentence)
  p = re.compile('[A-Z]{2,}[a-z]*')
  chunkNodes=getNounChunks(doc)
  defination=""
  for token in featureList:
    if p.match(str(token['token'])):
      term=str(token['token'])
      for chunks in chunkNodes:
        if term in str(chunks[3]):
#           print(term)
          defination=str(chunks[0])
          informationTerm=getPhraseSpan(sentence, getTokenIndex(sentence,token['token']))
          domain=str(informationTerm)[str(informationTerm).index(defination)+len(defination):]
          resultTerminology['key']=term
          resultTerminology['definition']=defination
          resultTerminology['reference']=domain
  return resultTerminology

In [0]:
def getAimTemplate(doc,sentence,featureList):
  aimDict={"problem":'',"domain":'',"hypothesis":''}
  aimVerbList=['aim','aimed','take','took','train','trained','direct','directed','purpose','purposed','purport','purported','drive','drove','get','got','calculate','calculated','target','targeted','place','placed','aspire','aspired']
  aimNounList=['purpose','intent','intention','design','object','bearing','heading','target']
  positiveFlag = False
  doc=nlp(sentence)
  for index, feature in enumerate(featureList):
    if (feature["lemma"] in aimVerbList) or (feature["lemma"] in aimVerbList):
      positiveFlag=True
      if not positiveFlag:
        return resultDict
      rootToken = getRoot(sentence)
      verbSet = getVerbWithSubject(sentence)
      rootIndex = getTokenIndex(sentence, rootToken)
      allChildrenList=getAllChildren(sentence,rootIndex)
      for x in allChildrenList:
        if x.dep_=='prep':
          aimDict['domain']=getPhraseSpan(sentence,getTokenIndex(sentence,x))
        if str(aimDict['domain'])=='' and x.dep_=='nsubj':
          aimDict['domain']=getPhraseSpan(sentence,getTokenIndex(sentence,x))
        if x.dep_=='xcomp':
          for y in getAllChildren(sentence,getTokenIndex(sentence,x)):
            if y.dep_=='prep':
              aimDict['hypothesis']=str(getPhraseSpan(sentence,getTokenIndex(sentence,y)))
            if y.dep_=='dobj':
              aimDict['problem']=x.text+" "+str(getPhraseSpan(sentence,getTokenIndex(sentence,y)))
              
  if not positiveFlag:
    return aimDict
  
  
  
  return aimDict

In [0]:
def getObservationTemplate(doc,sentence,featureList):
  observationDict={"object":'',"action":'',"observation":'',"hypothesis":''}
  observationVerbList={'observe','modify','consider','be','make','transfer'}
  doc=nlp(sentence)
  chunkNodeList=getNounChunks(doc)
  for index, feature in enumerate(featureList):
    if str(feature['lemma']) in observationVerbList:
      if str(feature['dependency'])=='ROOT':
        action=feature['token']
        observationDict['action']=action
        childrenList=getAllChildren(sentence,getTokenIndex(sentence,action))
        for x in childrenList:
          if 'NN' in str(x.tag_):
            observationDict['observation']=getPhraseSpan(sentence,getTokenIndex(sentence,x))
          if str(x.tag_)=='IN' or str(x.tag_)=='VBG':
            observationDict['hypothesis']=getPhraseSpan(sentence,getTokenIndex(sentence,x))
          for chunks in chunkNodeList:
            if 'subj' in str(chunks[2]):
              observationDict['object']=str(getPhraseSpan(sentence,getTokenIndex(sentence,chunks[0])))
              if len(observationDict['object']) < len(str(chunks[1])):
                observationDict['object']=str(chunks[0])
           
  return observationDict

In [0]:
def getSelectionTemplate(doc,sentence,featureList):
  selectionDict={"sample":'',"stage":'',"sample criteria":'',"justification":''}
  selectionVerbList={'select'}
  doc=nlp(sentence)
  chunkNodeList=getNounChunks(doc)
  passive=False
  active=False
  for index,feature in enumerate(featureList):
    if str(feature['lemma']) in selectionVerbList:
      if str(feature['dependency'])=='ROOT':
        childrenList=getAllChildren(sentence,getTokenIndex(sentence,feature['token']))
        for chunks in chunkNodeList:
          if ('subjpass' in str(chunks[2])) and (str(feature['lemma']) in str(chunks[3])):
            selectionDict['sample']=chunks[0]
          if ('nsubj'== str(chunks[2])) and (str(feature['lemma']) in str(chunks[3])):
            selectionDict['stage']=chunks[0]
            
        for child in childrenList:
          for chunks in chunkNodeList:
            if 'subjpass' in str(chunks[2]):
              passive=True
            if 'nsubj'==str(chunks[2]):
              active=True
            if passive and (str(child)== str(chunks[1])):
              childSubList=getAllChildren(sentence,getTokenIndex(sentence,child))
              for childSub in childSubList:
                for chunksSub in chunkNodeList:
                  if ('pobj' in str(chunksSub[2])) and (str(childSub)== str(chunksSub[3])):
                    selectionDict['sample criteria']=str(getPhraseSpan(sentence,getTokenIndex(sentence,childSub)))
            elif passive and ('pobj' in str(chunks[2])) and (str(child)== str(chunks[3])):
              selectionDict['stage']=str(getPhraseSpan(sentence,getTokenIndex(sentence,child)))
            elif passive and ('dobj' in str(chunks[2])) and (str(child)== str(chunks[3])):
              selectionDict['justification']=str(getPhraseSpan(sentence,getTokenIndex(sentence,child)))
              
            if active and (str(child)== str(chunks[1])):
              childSubList=getAllChildren(sentence,getTokenIndex(sentence,child))
              for childSub in childSubList:
                for chunksSub in chunkNodeList:
                  if ('dobj' in str(chunksSub[2])) and (str(childSub)== str(chunksSub[3])):
                    selectionDict['sample criteria']=str(getPhraseSpan(sentence,getTokenIndex(sentence,childSub)))
            for indexSub,featureSub in enumerate(featureList):
              if active and ('dobj' in str(chunks[2])) and (str(feature['lemma'])== str(chunks[3])) and featureSub['pos']=='NOUN':
                selectionDict['sample']=str(getPhraseSpan(sentence,getTokenIndex(sentence,child)))
              elif active and ('dobj' in str(chunks[2])) and (str(feature['lemma'])== str(chunks[3])) and featureSub['pos']=='VERB':
                selectionDict['justification']=str(getPhraseSpan(sentence,getTokenIndex(sentence,child)))
                 
  return selectionDict

In [0]:
def getContributionTask(sentence, rootIndex):
  task = ""
  doc = nlp(sentence)
  #ckeft children for domain and task
  leftChildrenList = getAllChildren(sentence, rootIndex)
  for childToken in leftChildrenList:
    #skip punct
    if childToken.dep_ != 'punct' and childToken.dep_!='':

      #search for nsubj or nsubjpass
      if  childToken.dep_=='dobj' or childToken.dep_ == 'nsubjpass':
        taskChildIndex = getTokenIndex(sentence, childToken)
        if taskChildIndex!=-1:
          taskSpan = getPhraseSpan(sentence, taskChildIndex)
          task= taskSpan
          return task
      #if taskChildIndex -ends
      if childToken.dep_ == 'nsubj':
        taskChildIndex = getTokenIndex(sentence, childToken)
        if taskChildIndex!=-1:
          taskSpan = getPhraseSpan(sentence, taskChildIndex)
          task=taskSpan
          print("in getTask: ", task)
  return task

In [0]:
def getContributeTemplate(sentence, featureList):
  #initializing dictionary
  contributionDict = {"research-work":"", "researchers": "", "research Time":"", "research Institute":"", "relevance": ""}
  
  #initializing indecator list
  contributeVerbList = ['propose', 'conduct' "approach",'access' 'suggest','claim','nominate', 'present','contribute', 'add', 'bring','give','lead','thank', 'perform', 'execute','dedicate','devote','share','do', 'generate' 'explore','create', 'make', 'produce', 'support', 'design','work','plan','invent','formulate','study', 'identify']
  nounList = ["result", "outcome", "solution","effect",'trace', 'draw','discover', 'describe','report', "decision", "answer", "final result",  "fruit", "fruition", "task", "method","research", "technique", "key"]
  
  removeWordList = []
  researcherPronoun = ""
  taskThisWork = ""
  #check for indicator words
  positiveFlag = False
  for index, feature in enumerate(featureList):
    if feature["dependency"]=="poss":
      researcherPronoun = feature["token"]
    if feature["token"].text=="work":
      taskThisWork = getPhraseSpan(sentence, index).text
    if (feature["lemma"] in contributeVerbList) or (feature["lemma"] in nounList):
      positiveFlag=True
    #if -ends
  #for -ends
  
  if not positiveFlag:
    return contributionDict

  NERDict = getNERFeatures(doc)
  authorList = []
  for key in NERDict:
    if NERDict[key]=="PERSON":
      authorList.append(key)
      removeWordList.extend(key.lower().split())
    elif NERDict[key]=="DATE":
      contributionDict["research Time"]=key
      removeWordList.extend(key.lower().split())
    elif NERDict[key]=="ORG" or NERDict[key]=="GPE":
      contributionDict["research Institute"]+=" "+key  
      removeWordList.extend(key.lower().split())
  #for key -ends
  
  
  if len(authorList)!=0:
    contributionDict["researchers"]=authorList
  elif not (isinstance(researcherPronoun, str)):
    contributionDict["researchers"] = [researcherPronoun.text]
  else:
    contributionDict["researchers"]=authorList
 
  
  #get root
  rootToken = getRoot(sentence)
  rootIndex = getTokenIndex(sentence, rootToken)
  verbSet = getVerbWithSubject(sentence)
  
  taskSpan = getContributionTask(sentence, rootIndex) 
  
  if (isinstance(taskSpan, str))  and ("this".upper() in taskThisWork.upper()):
    contributionDict["research-work"]= taskThisWork  
    task = taskThisWork
  elif (isinstance(taskSpan, str))  and ("this".upper() not in taskThisWork.upper()):
    contributionDict["research-work"]= taskSpan
    task = taskThisWork

  elif (str(taskSpan.root.pos_)=='PRON') and ("this".upper() in taskThisWork.upper()):
    contributionDict["research-work"]= taskThisWork  
    task = taskThisWork
  else:
    contributionDict["research-work"]= taskSpan.text
    task = taskSpan.text
  removeWordList.extend(task.lower().split())

  
  relevanceSentence = removeWords(sentence, removeWordList)
  
  relevanceDoc = nlp(relevanceSentence)
  nounChunkList = getNounChunks(relevanceDoc)
  for chunk in nounChunkList:
    if chunk[2]=="pobj":
      contributionDict["relevance"]=chunk[0]
    #if -ends
  #for -ends

  return contributionDict
  
  

In [0]:
def getProcessTemplate(sentence, featureList):
  #initializing dictionary
  processDict = {"method name":"", "purpose": "", "process description":"", "assumptions":"", "tool": ""}
  
  #initializing indecator list
  contributeVerbList = ['propose', 'conduct' "approach",'access', 'assess', "perform", 'suggest','claim', 'use', 'monitor', 'draw','discover', 'describe','utilize','apply', 'practice','nominate', 'present','contribute', 'add', 'bring','give','lead', 'perform', 'execute','devote','share','do', 'generate' 'explore','create', 'make', 'produce', 'support', 'design','work','plan','invent','formulate','study', 'identify']
  nounList = ["result", "task", "monitor", "category", "input", "output", "tool", "frameWork" "project", "work", "network","system","treatment","protocol","symptoms", "outcome", "solution","effect",'trace', 'report', "decision", "answer", "final result",  "fruit", "fruition", "task", "method","research", "technique", "key"]
  
  removeWordList = []

  #check for indicator words
  positiveFlag = False
  for index, feature in enumerate(featureList):
    if (feature["lemma"] in contributeVerbList) or (feature["lemma"] in nounList):
      positiveFlag=True
    #if -ends
  #for -ends
  if not positiveFlag:
    return processDict
 
  #get root
  rootToken = getRoot(sentence)
  rootIndex = getTokenIndex(sentence, rootToken)
  verbSet = getAllVerbs(sentence)
  
  #get chunks
  doc = nlp(sentence)
  chunkList = getNounChunks(doc)
  
  for chunk in chunkList:
    #process name is subject
    if chunk[2]=='nsubj' or chunk[2]=='nsubjpass':
      processDict["method name"]=getTask(sentence, rootIndex)
      removeWordList.extend(processDict["method name"].lower().split())
    #if for method name -emds
    if chunk[3].lower()=='using' or chunk[3].lower()=='with' or chunk[3].lower()=='by' or chunk[3].lower()=='via' or chunk[3].lower()=='through':
      tokenIndex = getTokenIndex(sentence, chunk[3])
      processDict['tool']= getPhraseSpan(sentence, tokenIndex).text
      removeWordList.extend(processDict["tool"].lower().split())
      
    #if foe tool -ends
    if chunk[3].lower()=="for" or chunk[3].lower()=="on" or chunk[3].lower()=="that" or chunk[3].lower()=="in":
      processDict["purpose"]=chunk[0]
      removeWordList.extend(processDict["purpose"].lower().split())

    #if for purpose-ends
  #for -ends
  
  processSpan =""
  for verb in verbSet:
    #check whether method subject is part of verb or not? WHY? Gut feeling
    if verb.dep_!="ROOT" and verb.text!="using":
      verbIndex = getTokenIndex(sentence, verb)
      childrenList = getAllChildren(sentence, verbIndex)
      if len(childrenList)==0:
        continue
      processSpan = getPhraseSpan(sentence, verbIndex)
    #if -ends
   
    if isinstance(processSpan, str):
      processDict["process description"]=processSpan
    else:      
      processDict["process description"]=processSpan.text
  #for -ends
  removeWordList.extend(processDict["process description"].lower().split())
  
  relevanceSentence = removeWords(sentence, removeWordList)
  
  relevanceDoc = nlp(relevanceSentence)
  nounChunkList = getNounChunks(relevanceDoc)
  for chunk in nounChunkList:
    if chunk[2]=="pobj":
      processDict["assumptions"]=chunk[0]
    #if -ends
  #for -ends


  return processDict
  


In [0]:
def getConclusionTemplate(sentence, featureList):
  #initializing dictionary
  conclusionDict = {"task":"", "result": "", "achievement":"", "limitations":"", "futureWork": False}
  
  #initializing indecator list
  conclusionVerbList = ["yield", "give", "render","return", "provide", "generate", "prove", "predict", "allow", "permit", "foretell", "produce", "lead", "forcast", "report", "achieve", "accomplish", "reach", "contain"]
  conclusionNounList = ["result", "outcome", "analysis", "solution","evidence","proof","insight","effect", "decision", "answer", "final result", "score", "AP","precision", "recall", "accuracy", "fruit", "fruition", "r"]
  
  verbList = []
  #check for indicator words
  positiveFlag = False
  negationToken =""
  for index, feature in enumerate(featureList):
    if (feature["lemma"] in conclusionVerbList) or (feature["lemma"] in conclusionNounList):
      positiveFlag=True
    if (feature["lemma"] in conclusionVerbList):
      verbList.append(feature["token"].text)
    if (feature["dependency"]=='neg'):
      negationToken = feature["token"]
    #if -ends
  #for -ends
  if not positiveFlag:
    return conclusionDict
 
  #get root
  rootToken = getRoot(sentence)
  rootIndex = getTokenIndex(sentence, rootToken)
  verbSet = getAllVerbs(sentence)
  
  #get chunks
  doc = nlp(sentence)
  chunkList = getNounChunks(doc)
  
  #achievement
  print("verbSet", verbSet)
  print("verbList",verbList)
  for verb in verbSet:
    if verb.text in verbList:
      achievementdIndex = getTokenIndex(sentence, verb)
      achievementdSpan = getPhraseSpan(sentence, achievementdIndex)
      conclusionDict["achievement"]=achievementdSpan
      break
  
  #task
  subjectList = getAllSubject(sentence)
  taskSpan = ""
  print("subjectList:", subjectList)
  for subject in subjectList:
    if subject.pos_!='PRON':
      taskChildIndex = getTokenIndex(sentence, subject)
      taskSpan = getPhraseSpan(sentence, taskChildIndex)
    #if -ends
  #for -ends
  if isinstance(taskSpan, str):  
    task = getTask(sentence, rootIndex)
    print("task: ", task)
  else:
    task = taskSpan.text
  #if -ends
  
  conclusionDict["task"]=task
  
  #result
  print("chunkList for future",chunkList)
  for chunk in chunkList:
    if chunk[2]=='dobj':
      conclusionDict["result"]=chunk[0]
    elif chunk[2]=='pobj':
      conclusionDict["result"]=chunk[0]
    #if -ends
    break
   #for -ends
  
  
  
  #future
  for chunk in chunkList:
    if "future" in str(chunk[0]).lower() or "future" in str(chunk[3]).lower():
      conclusionDict["futureWork"]=True

  
  verbSubject = getVerbSubject(sentence, rootIndex)
  print("verbSubjectList", verbSubject)
  
  #limitations : find negation
  if not isinstance(negationToken, str):
    ancestorList = getAncestors(sentence, negationToken)
    print(ancestorList)
    ancestorIndex = getTokenIndex(sentence, ancestorList[0])
    conclusionDict["limitations"]=getPhraseSpan(sentence,ancestorIndex)
    
    
    
  
  return conclusionDict
  


In [0]:
def getComparisionTemplate(sentence, featureList):
  
  removeWordList = []

  #initializing dictionary
  compareDict = {"subject1":"", "subject2": "", "domain":"", "outcome":""}
  
  #initializing indecator list
  indicationNounList = ["than", "as", "like","more", "less"]
    
  #check for indicator words
  positiveFlag = False
  indicationToken=""
  for index, feature in enumerate(featureList):
    if (feature["lemma"] in indicationNounList):
      positiveFlag=True
      indicationToken=feature["token"]
    #if -ends
  #for -ends
  print("indicationToken", indicationToken)
  
  #check for degree - er or est
  degreeWordList=re.findall(r'\b(\w+(er|est))\b',sentence)
  
  
  if len(degreeWordList)>0:
    positiveFlag=True
    compareDict["outcome"]=degreeWordList[0][0]
    removeWordList.extend(compareDict["outcome"].lower().split())

    
  if not positiveFlag:
    return compareDict
 
  #get root
  rootToken = getRoot(sentence)
  rootIndex = getTokenIndex(sentence, rootToken)
  verbSet = getAllVerbs(sentence)
  
  #get chunks
  doc = nlp(sentence)
  chunkList = getNounChunks(doc)
  
  #subject 1
  
  subjectList = getAllSubject(sentence)
  taskSpan = ""
  print("subjectList:", subjectList)
  for subject in subjectList:
    if subject.pos_!='PRON':
      taskChildIndex = getTokenIndex(sentence, subject)
      taskSpan = getPhraseSpan(sentence, taskChildIndex)
    #if -ends
  #for -ends
  compareDict["subject1"]=taskSpan.text
  removeWordList.extend(compareDict["subject1"].lower().split())

    
  #subject 2
  indicationTokenIndex= getTokenIndex(sentence, indicationToken)
  for chunk in chunkList:
    if not isinstance(indicationToken, str):
      if chunk[3]==indicationToken.text:
        compareDict["subject2"]=chunk[0]
  
  removeWordList.extend(compareDict["subject2"].lower().split())

  relevanceSentence = removeWords(sentence, removeWordList)
  
  relevanceDoc = nlp(relevanceSentence)
  nounChunkList = getNounChunks(relevanceDoc)
  for chunk in nounChunkList:
    if chunk[2]=="pobj":
      compareDict["domain"]=chunk[0]
    #if -ends
  #for -ends
  

  
  return compareDict
  


In [0]:
'''Acknowledgement'''
#sentence = "Personal diet management is key to fighting the obesity epidemic."
sentence="A. Geiger, P. Lenz, and R. Urtasun ; “Are we ready for autonomous driving? the kitti vision benchmark suite,”; Computer Vision and Pattern Recognition (CVPR), 2012 IEEE Conference;  2012; pp. 3354–3361"
#sentence="B. Li, T. Zhang, and T. Xia, “Vehicle detection from 3d lidar using fully convolutional network,” in Proceedings of Robotics: Science and Systems, AnnArbor, Michigan, June 2016."

'''results'''
# sentence = "For LID, the communication task yielded the best results detection: AUC = 0.930."
# sentence =  "For the car class, our RPN variants achieve a 91% recall at just 50 proposals, whereas MV3D [4] reported requiring 300 proposals to achieve the same recall."
# sentence = "As an example, our Feature Pyramid based fusion RPN achieves an 86% 3D recall on the car class with just 10 proposals per frame."

'''contribution'''
'''working '''
# sentence = "information extraction on research paper was performed by Nidhi Vaishnav and Maitri Shah in December 2018 at University of Texas at Dallas under natural language processing subject."
# sentence = "Sir Issac Newton performed gravity experiments in 17th century at England which initiated kinematics."
# sentence = "Presenting one of the key work in the area of 3D forensics, Kanchan Bahirat identified various possible attacks on 3D LiDAR data as well as two forgery detection methods based on consistency in the key characteristics of LiDAR data such as: sampling density and occlusion effect during 2016."
# sentence = "Our study employed data from a cross-sectional survey and physical examinations of twins in a population-based twin registry, the Kaiser Permanente Women Twins Study Examination II, conducted in 1989 to 1990 in Oakland, California, United States."
'''accaptable'''
# sentence = "This work was begun and brought to fruition when we were at Cambridge University; we dedicate this paper to the memory of Professor David G. Crighton, who first interested one of us (L.M.) in this problem nearly a decade ago."
# sentence = "This work was supported by the European Community through Marie-Curie Fellowship HPMF-2002-01915 (to M.A.) and the U.S. Office of Naval Research through a Young Investigator Award (to L.M.)."
# sentence = "Wang et al [19] described another approach where after one wavelet decomposition, the norm and orientation of each wavelet coefficient vector are modified independently."
# sentence = "Among the early work, Ohbuchi [23] generated a non-manifold mesh from a point cloud and performed a mesh spectral analysis for embedding watermark into spectral coefficients."
# sentence = "Cotting designed a technique that: (a) partitions a point cloud data into patches; (b) transforms them into discrete frequency bands; (c) embeds a watermark into the lowfrequency components."

'''process'''
sentence = "a new interactive mobile system enables automated heart assessment based on walking and sleep cycle of user using heart rate monitor."
# sentence="Using these proposals, the second stage detection network performs accurate oriented 3D bounding box regression and category classification to predict the extents, orientation, and classification of objects in 3D space with help of Python language."
# sentence="In addition to using techniques in computer vision and machine learning, one unique feature of this system is the realization of real-time energy balance monitoring through metabolic network simulation."
# sentence="Movement trajectories of individual joints were extracted from videos of PD assessment using Convolutional Pose Machines, a pose estimation algorithm built with deep learning."

''' conclusion '''
'''good'''
# sentence = "Our analysis gave good results for templates. we achieved 75% accuracy, but it doesn't work on discourse level. We will do discourse in future"
# sentence="Our analysis allows us to predict a critical speed for the onset of flapping as well as the frequency of flapping. But it doesn't give good critical speed for lightning bolt"
# sentence="The proposed system provides insight into the potential of computer vision and deep learning for clinical application in PD and demonstrates promising performance for the future translation of deep learning to PD clinical practices."
'''accaptable'''
# sentence="These results provide novel evidence that lifetime socioeconomic position influences adult health and highlight the utility of studying social plus biological aspects of twinship."
# sentence="Our proposed architecture is shown to produce state of the art results on the KITTI 3D object detection benchmark [1] while running in real time with a low memory footprint, making it a suitable candidate for deployment on autonomous vehicles."

# ----------------------------------------------------------------Terminology-----------------------------------------------------------------------------------#


# sentence="When considering sparse and low resolution input such as the Front View or BEV, Bird’s Eye View point cloud projections, this method is not guaranteed to have enough information to generate region proposals, especially for small object classes."
#  sentence="In this paper, we aim to resolve these difficulties by proposing AVOD, an Aggregate View Object Detection architecture for autonomous driving."
# sentence="Nine participants with PD and LID completed a levodopa infusion protocol, where symptoms were assessed at regular intervals using the Unified Dyskinesia Rating Scale (UDysRS) and Unified Parkinson's Disease Rating Scale (UPDRS). "
# sentence="Features of the movement trajectories (e.g. kinematic, frequency) were used to train random forests to detect and estimate the severity of parkinsonism and LID."
##show
# sentence="We present AVOD, an Aggregate View Object Detection network for autonomous driving scenarios."
# sentence="AR is Augment Reality"



#------------------------------------------------------------------Aim---------------------------------------------------------------------------------------#
## show
# sentence="In this paper, we aim to resolve these difficulties by proposing AVOD, an Aggregate View Object Detection architecture for autonomous driving."
# sentence="The purpose of information extraction is to make easy the question answer process by running the seekers project"
# sentence="This study aims to put forth a new method by using georeferenced community-authored reviews for restaurants."
# sentence="This study aims to contribute this line of studies by proposing taste as an indicator of social status which integrates different facets of culture, economy and social networks of urban inhabitants. "
# sentence="The objective of this paper aims to compute xyx by using first method"


#--------------------------------------------------------------Observation--------------------------------------------------------------------------------------#
# sentence="Persons with PD and their physicians must regularly modify treatment regimens and timing for optimal relief of symptoms."
##show
# sentence="Computer vision is an attractive solution for automated assessment of PD, by making possible recent advances in computational power and deep learning algorithms."
# sentence="The remarkable progress made by deep neural networks on the task of 2D object detection in recent years has transferred well to the detection of objects in 3D."
# sentence="RPNs can be considered a weak amodal detector, providing proposals with high recall and low precision."


#-----------------------------------------------------------Selection-----------------------------------------------------------------------#
# sentence="We then select those features which determine one’s choice of restaurant and the rating that he/she provides for that restaurant by using best method."
##show
# sentence="Forty-five features of the three categories (i.e., foods and drinks, food adjectives, and ambience adjectives) were selected at this step using xyz method."

# sentence = "spacy is better than nltk for natural language processing task"
# sentence = "2D image detection is better than 3D image detection for Kitti dataset."
# sentence = "3D image detection is not so good as 2D image detection for ILSVRC dataset."
# sentence="We accordingly compared the health status among monozygotic and dizygotic women twin pairs who lived together through childhood (until at least age 14) and subsequently were discordant or concordant on adult socioeconomic position. This comparison permitted us to ascertain the additional impact of adult experiences on adult health in a population matched on early life experiences."


'accaptable'

In [0]:
featureList = []

tokens = getTokens(sentence)
print(tokens)

doc = nlp(sentence)

featureList = getFeatures(doc, featureList)
print(len(featureList))

for i in featureList:
  print(i)
#for -ends


['a', 'new', 'interactive', 'mobile', 'system', 'enables', 'automated', 'heart', 'assessment', 'based', 'on', 'walking', 'and', 'sleep', 'cycle', 'of', 'user', 'using', 'heart', 'rate', 'monitor', '.']
21
{'token': a, 'lemmaVal': 11901859001352538922, 'lemma': 'a', 'pos': 'DET', 'tag': 'DT', 'shape': 'x', 'isAlpha': True, 'isStop': True, 'dependency': 'det', 'headText': 'system', 'headTag': 'NOUN', 'children': []}
{'token': new, 'lemmaVal': 4753564829687343602, 'lemma': 'new', 'pos': 'ADJ', 'tag': 'JJ', 'shape': 'xxx', 'isAlpha': True, 'isStop': False, 'dependency': 'amod', 'headText': 'system', 'headTag': 'NOUN', 'children': []}
{'token': interactive, 'lemmaVal': 9613289368394214984, 'lemma': 'interactive', 'pos': 'ADJ', 'tag': 'JJ', 'shape': 'xxxx', 'isAlpha': True, 'isStop': False, 'dependency': 'amod', 'headText': 'system', 'headTag': 'NOUN', 'children': []}
{'token': mobile, 'lemmaVal': 13895322422246515550, 'lemma': 'mobile', 'pos': 'ADJ', 'tag': 'JJ', 'shape': 'xxxx', 'isAlpha':

In [0]:
# print("Final:{}".format(wordnetFeatures(tokens)))
wordNetFeatureDict=wordnetFeatures(tokens)
print(wordNetFeatureDict)

['a', 'new', 'interactive', 'mobile', 'system', 'enables', 'automated', 'heart', 'assessment', 'based', 'on', 'walking', 'and', 'sleep', 'cycle', 'of', 'user', 'using', 'heart', 'rate', 'monitor', '.'] 

{'a': {'hypernym': ['metric_linear_unit', 'fat-soluble_vitamin', 'nucleotide', 'purine', 'current_unit', 'letter', 'blood_group'], 'hyponym': ['vitamin_A1', 'vitamin_A2'], 'holonym': ['nanometer', 'abampere'], 'meronym': ['picometer', 'milliampere'], 'entail': [], 'synonym': ['angstrom', 'angstrom_unit', 'A', 'vitamin_A', 'antiophthalmic_factor', 'axerophthol', 'A', 'deoxyadenosine_monophosphate', 'A', 'adenine', 'A', 'ampere', 'amp', 'A', 'A', 'a', 'A', 'type_A', 'group_A']}, 'new': {'hypernym': [], 'hyponym': [], 'holonym': [], 'meronym': [], 'entail': [], 'synonym': ['new', 'fresh', 'new', 'novel', 'raw', 'new', 'new', 'unexampled', 'new', 'new', 'newfangled', 'new', 'New', 'Modern', 'New', 'new', 'young', 'new', 'newly', 'freshly', 'fresh', 'new']}, 'interactive': {'hypernym': [], 

In [0]:
ackTemplate = getAcknowledgementTemplate(doc, sentence)
print(ackTemplate)

{'authors': [], 'paper': '', 'publication': '', 'publish-year': '', 'reference-Page': ''}


In [0]:
resultDict = getResultTemplate(doc, sentence, featureList)
print(resultDict)

{'domain': '', 'task': '', 'result': ''}


In [0]:
getContributeTemplate(sentence, featureList)

{'relevance': '',
 'research Institute': '',
 'research Time': '',
 'research-work': '',
 'researchers': ''}

In [0]:
getProcessTemplate(sentence, featureList)

{'assumptions': 'user monitor',
 'method name': 'a new interactive mobile system',
 'process description': 'walking and sleep',
 'purpose': 'walking and sleep cycle',
 'tool': 'using heart rate monitor'}

In [0]:
getConclusionTemplate(sentence, featureList)

{'achievement': '',
 'futureWork': False,
 'limitations': '',
 'result': '',
 'task': ''}

In [0]:
print(getTerminologyTemplate(doc,sentence,featureList))

{'key': '', 'definition': '', 'reference': ''}


In [0]:
print(getAimTemplate(doc,sentence,featureList))

{'problem': '', 'domain': '', 'hypothesis': ''}


In [0]:
print(getObservationTemplate(doc,sentence,featureList))

{'object': '', 'action': '', 'observation': '', 'hypothesis': ''}


In [0]:
print(getSelectionTemplate(doc,sentence,featureList))

{'sample': '', 'stage': '', 'sample criteria': '', 'justification': ''}


In [0]:
getComparisionTemplate(sentence, featureList)

indicationToken 
subjectList: [system]


{'domain': 'walking and sleep cycle',
 'outcome': 'user',
 'subject1': 'a new interactive mobile system',
 'subject2': ''}

In [0]:
displacy.render(doc, style='dep', jupyter=True, options={'distance':125})